# Single-Pass Tool-Using Helpdesk Agent (OpenAI Version)

In this notebook, we do a **single call** to OpenAI’s API to decide whether the agent:
1. **Invokes exactly one** tool (user directory, knowledge base, or ticketing), or
2. Provides a **final answer** without tool usage.

**Tools** are **dummy**—they only return a mock message or data. The OpenAI response is used to parse out either a `TOOL_ACTION` or `FINAL_ANSWER`. No iterative loop or multiple LLM calls.

## 1. Install and Import Dependencies
Make sure you have the `openai` library installed:
```
pip install openai
```
Also, set your **OpenAI API key** in an environment variable `OPENAI_API_KEY` or configure it in your code (not shown here for security).

In [ ]:
from openai import OpenAI
import os
import re

os.environ['OPENAI_API_KEY']=''

# Optionally set your API key in code (not recommended for production)
# openai.api_key = "YOUR_OPENAI_API_KEY"  # Typically better to set via environment.

## 2. Define Dummy Tools
We create **three** placeholder functions, each returning a **mock** response.

In [ ]:
def user_directory_tool(user_id):
    """
    Mock function to get user info from an IAM system.
    """
    return {
        "user_id": user_id,
        "account_status": "locked",
        "last_login": "2025-02-15 10:00 AM"
    }

def knowledge_base_tool(query):
    """
    Mock function to search a knowledge base.
    """
    kb_articles = {
        "email lockout": "To resolve an email lockout, confirm the user's identity, reset the password, and unlock the account.",
        "password reset": "Users can reset their password by visiting the reset portal or calling the helpdesk."
    }
    for keyword, article in kb_articles.items():
        if keyword in query.lower():
            return article
    return "No relevant article found."

def ticketing_tool(action, description):
    """
    Mock function to create or close a support ticket.
    """
    if action == "create":
        return f"Ticket created with description: '{description}'."
    elif action == "close":
        return f"Ticket closed with note: '{description}'."
    else:
        return f"Unknown ticket action '{action}'."

## 3. OpenAI LLM Call
We'll define a function `call_openai_for_action` that:
1. Sends the user's **query** to OpenAI.
2. **Instructs** the model to either produce `TOOL_ACTION:` (with details) or `FINAL_ANSWER:`.
3. Returns the raw text from the LLM.

You can tweak the system/user messages and the prompt structure to your preference.

In [ ]:
def call_openai_for_action(user_query, temperature=0.2):
    """
    Calls the OpenAI API once to decide which tool to invoke or finalize an answer.
    We expect a response containing either:
      TOOL_ACTION: <tool_name>, param='value', ...
      or
      FINAL_ANSWER: <some text>
    """

    system_message = (
        "You are a helpdesk AI. You have 3 possible tools to help you:"+
        "1) user_directory (requires user_id='xxx')"+
        "2) knowledge_base (requires query='some string')"+
        "3) ticketing (requires action='create' or 'close', description='string')"+
        "Decide which tool to use based on the user query, or provide a final answer."+
        "Format your reply as either:"+
        "TOOL_ACTION: <tool_name>, key='value', ..."+
        "or"+
        "FINAL_ANSWER: <text>"
    )

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_query}
    ]

    client = OpenAI()

    response = client.chat.completions.create(
        model="gpt-4o",  # or 'text-davinci-003', depending on your plan
        messages=messages,
        temperature=temperature,
        max_tokens=200,
    )

    raw_text = response.choices[0].message.content
    print(raw_text)
    return raw_text

## 4. Single-Pass Helpdesk Agent
We define a class that:
1. Receives a user query.
2. Makes a **single call** to `call_openai_for_action(...)`.
3. If the returned text starts with `TOOL_ACTION:`, it parses out the tool name and parameters, calls the dummy tool, and returns the observation.
4. If it starts with `FINAL_ANSWER:`, it just returns that.
5. If neither is found, a fallback message is given.

In [ ]:
class HelpdeskToolAgent:
    def __init__(self):
        self.conversation_log = []

    def handle_user_query(self, user_query: str):
        # Log user query
        self.conversation_log.append(f"User: {user_query}")

        # Call OpenAI to decide on a tool or final answer
        llm_response = call_openai_for_action(user_query)
        self.conversation_log.append(f"LLM: {llm_response}")

        # Check for TOOL_ACTION or FINAL_ANSWER
        if "TOOL_ACTION:" in llm_response:
            action_str = llm_response.split("TOOL_ACTION:")[-1].strip()
            return self._invoke_tool(action_str)

        elif "FINAL_ANSWER:" in llm_response:
            return llm_response.split("FINAL_ANSWER:")[-1].strip()

        # fallback
        return "Could not parse a tool action or final answer."

    def _invoke_tool(self, action_str):
        # We expect something like: "user_directory, user_id='U9999'"
        # or: "ticketing, action='create', description='...'"

        # Let's parse out the tool name first
        parts = action_str.split(",")
        tool_name = parts[0].strip()
        # The rest are key='value' pairs
        kwargs = {}
        if len(parts) > 1:
            # e.g. key='value', key2='value2'
            param_str = ",".join(parts[1:])
            matches = re.findall(r"(\w+)='([^']*)'", param_str)
            for k, v in matches:
                kwargs[k] = v

        # Call the corresponding dummy tool
        if tool_name == "user_directory":
            result = user_directory_tool(kwargs.get("user_id", ""))
            return f"Tool invoked: {tool_name}\nResult: {result}"
        elif tool_name == "knowledge_base":
            result = knowledge_base_tool(kwargs.get("query", ""))
            return f"Tool invoked: {tool_name}\nResult: {result}"
        elif tool_name == "ticketing":
            result = ticketing_tool(
                action=kwargs.get("action", ""),
                description=kwargs.get("description", "")
            )
            return f"Tool invoked: {tool_name}\nResult: {result}"
        else:
            return f"Unknown tool '{tool_name}'."

## 5. Main Orchestration
We create the agent, provide a **sample** user query (e.g., “I’m locked out of my email”), and print the **single** response. The LLM might say “TOOL_ACTION: user_directory, user_id='U9999'” or “FINAL_ANSWER: ...”

In [ ]:
if __name__ == "__main__":
    agent = HelpdeskToolAgent()

    sample_query = "Hi, I'm locked out of my email account. How can I fix this?"
    response = agent.handle_user_query(sample_query)
    print("=== SINGLE-PASS HELPDESK RESPONSE ===")
    print(response)

## Usage Notes
1. **API Key**: Make sure you have `OPENAI_API_KEY` set as an environment variable (or set `openai.api_key` in code). Without it, the call will fail.
2. **Model**: We use `gpt-3.5-turbo` in `openai.ChatCompletion.create`. If you have access to GPT-4, you can switch to `gpt-4`. Or if you prefer a completion model, adapt the code accordingly.
3. **Prompt Engineering**: The system prompt instructs the model to choose a tool from `[user_directory, knowledge_base, ticketing]` or provide a final answer.
4. **Dummy Tools**: We only show mock returns from `user_directory_tool`, `knowledge_base_tool`, `ticketing_tool`. In reality, you’d integrate actual APIs.
5. **Single Pass**: This approach does exactly **one** LLM call, so it won't do multiple steps or reevaluate after tool usage. If you need that, consider a multi-step agent with memory or reflection.

## Example Output
A typical run (with certain prompt/response) might yield:
```
=== SINGLE-PASS HELPDESK RESPONSE ===
Tool invoked: user_directory
Result: {'user_id': 'U9999', 'account_status': 'locked', 'last_login': '2025-02-15 10:00 AM'}
```
indicating that the LLM recommended we check the user directory for a locked account. Alternatively, it might produce a final answer saying no tool is needed if it doesn’t see relevant keywords.

## Key Takeaways
1. **OpenAI Integration**: This code makes a single call to `openai.ChatCompletion.create(...)`.
2. **One Tool / One Response**: We parse out `TOOL_ACTION:` from the LLM's text and call the corresponding function. Otherwise, we return the `FINAL_ANSWER:`.
3. **Expandable**: In production, you might keep a **registry** of available tools, handle more complex parameters, or incorporate multi-step logic if needed.